<a href="https://colab.research.google.com/github/bigcarrot0722/kobert_emotion_analysis/blob/main/Model_Test/Seven_Feeling_Model_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Download Resource

In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece==0.1.91
!pip install transformers==4.8.2
!pip install torch

In [ ]:
pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

In [ ]:
from kobert_tokenizer import KoBERTTokenizer 
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
tokenizer.encode("한국어 모델을 공유합니다.")

In [ ]:
import torch
from transformers import BertModel

In [ ]:
def get_kobert_model(model_path, vocab_file, ctx="cpu"):
    bertmodel = BertModel.from_pretrained(model_path, return_dict=False)
    device = torch.device(ctx)
    bertmodel.to(device)
    bertmodel.eval()
    vocab_b_obj = nlp.vocab.BERTVocab.from_sentencepiece(vocab_file,
                                                         padding_token='[PAD]')
    return bertmodel, vocab_b_obj

In [ ]:
import gluonnlp as nlp
bertmodel, vocab = get_kobert_model('skt/kobert-base-v1',tokenizer.vocab_file)

Downloading:   0%|          | 0.00/535 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/369M [00:00<?, ?B/s]

## Drive Mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/PBL')

## Test Set

In [ ]:
import pandas as pd
feeling = pd.read_excel('feeling2.xlsx')

In [ ]:
feeling.loc[(feeling['Emotion'] == '공포'), 'Emotion'] = 0
feeling.loc[(feeling['Emotion'] == '슬픔'), 'Emotion'] = 1
feeling.loc[(feeling['Emotion'] == '중립'), 'Emotion'] = 2
feeling.loc[(feeling['Emotion'] == '놀람'), 'Emotion'] = 3
feeling.loc[(feeling['Emotion'] == '행복'), 'Emotion'] = 4
feeling.loc[(feeling['Emotion'] == '혐오'), 'Emotion'] = 5
feeling.loc[(feeling['Emotion'] == '분노'), 'Emotion'] = 6

In [ ]:
data_list = []

for a, label in zip(feeling['Sentence'], feeling['Emotion']):
  data = []
  data.append(a)
  data.append(str(label))

  data_list.append(data)

In [ ]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data_list, test_size = 0.2, random_state = 0)
test_data, valid_data = train_test_split(test_data, test_size = 0.5, random_state = 0)

## Test Set 형식 맞추기 & Model load에 필요한 것들 선언하기

In [ ]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
# BERT 모델에 들어가기 위한 dataset을 만들어주는 클래스
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# Setting parameters
max_len = 64
batch_size = 1
warmup_ratio = 0.1
num_epochs = 4
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
#토큰화
tok = tokenizer.tokenize

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [ ]:
data_valid = BERTDataset(valid_data, 0, 1, tok, vocab, max_len, True, False)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=1, num_workers=5)

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=5,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
device = torch.device("cuda:0")

## Accuracy

In [ ]:
#5가지 감정 정확도 check
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

### 아래 BERTClassifier를 넣은 이유는 2가지 모델을 불러오기 위함

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

## a_model은 2가지 감정(놀람공포)에 해당하는 모델 load
## b_model은 2가지 감정(분노혐오)에 해당하는 모델 load
## cal_accuracy_result는 7가지 감정 test하는 function

In [ ]:
a_model=torch.load('2가지 감정(놀람공포)-batch64+epoch5_0.79')

In [ ]:
b_model = torch.load('2가지 감정(분노혐오)_batch16_epoch3_0.624')

In [ ]:
def calc_accuracy_two(X):
    max_vals, max_indices = torch.max(X, 1)
    return max_indices

In [ ]:
def calc_accuracy_result(X,Y, token_ids, valid_length, segment_ids):
    max_vals, max_indices = torch.max(X, 1)
    if max_indices == 0:
      out = a_model(token_ids, valid_length, segment_ids) #a_model은 놀람과 공포
      feeling = calc_accuracy_two(out)
      if feeling == 0:
        max_indices = 0
      else:
        max_indices = 3
    elif max_indices == 3:
      out = b_model(token_ids, valid_length, segment_ids) #a_model은 놀람과 공포
      feeling = calc_accuracy_two(out)
      if feeling == 0:
        max_indices = 5
      else:
        max_indices = 6
    if max_indices == Y:
      train_acc = 1
    else:
      train_acc = 0
    return train_acc

### 7가지 감정 테스트 코드(카테고리화 -> 세분화)

## saved_model은 5가지 감정 분류에 해당하는 모델 load

In [ ]:
saved_model=torch.load('5가지 감정_batch64_epoch5_0.651')

In [ ]:
saved_model.eval()
test_acc = 0
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
  token_ids = token_ids.long().to(device)
  segment_ids = segment_ids.long().to(device)
  valid_length= valid_length
  label = label.long().to(device)
  out = saved_model(token_ids, valid_length, segment_ids)
  test_acc += calc_accuracy_result(out, label,token_ids, valid_length, segment_ids)
print(batch_id)
print("epoch {} test acc {}".format(1, test_acc / (batch_id+1)))